# NYC Weather Data
The NYC weather data is from the National Centers for Environmental Information, https://www.ncdc.noaa.gov/cdo-web/datasets/GHCND/stations/GHCND:USW00094728/detail. The data is from the weather station at `NY CITY CENTRAL PARK, NY US` from `1869-01-01` to `2023-02-27`. And from the dates we need weather data, from 2006-01-01 to 2021-12-31, we see that are no null values. So there's minimal processing besides converting the date string to datetime, filtering by a time range, and selecting the columns we want.

## Data Dictionary
The data dictionary is available in the link above under the `Available Data Types` in `Station Data Inventory, Access & History`. It will show that `TAVG`, `TMIN`, `TMAX` is average, minimum, and maximum temperature respectively.

## Purpose
The only columns I'll use for in weather data is average temperature, minimum temperature, maximum temperature, precipitation, and snow fall. I think these columns will be meaningful to see the correlation between crime rates in NYC and the variables.

In [2]:
from os.path import join

import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    count, 
    when, 
    lit,
    isnan,
    col,
    concat_ws,
    from_unixtime,
    unix_timestamp,
    to_timestamp,
    year, 
    month,
    dayofmonth,
    hour
)

pio.renderers.default = "iframe"

In [3]:
def read_weather_data(fname: str=join("..", "data", "USW00094728.csv")) -> pd.DataFrame:
    df = pd.read_csv(fname, parse_dates=["DATE"])
    subset_df = df[["DATE", "TAVG", "TMAX", "TMIN", "PRCP", "SNOW"]]
    timerange = subset_df.query("DATE.between('2006-01-01', '2021-12-31')")
    return timerange.reset_index(drop=True)

In [4]:
df = read_weather_data()
df

,DATE,TAVG,TMAX,TMIN,PRCP,SNOW
0,2006-01-01,4.93,8.64,1.23,126.8,51.0
1,2006-02-01,2.07,5.78,-1.63,73.1,683.0
2,2006-03-01,6.15,10.33,1.98,20.3,33.0
3,2006-04-01,13.15,18.48,7.82,141.2,3.0
4,2006-05-01,17.27,21.97,12.57,117.5,0.0
...,...,...,...,...,...,...
187,2021-08-01,25.30,28.81,21.80,262.1,0.0
188,2021-09-01,21.27,24.79,17.76,254.8,0.0
189,2021-10-01,16.67,19.93,13.41,133.7,0.0
190,2021-11-01,7.91,11.54,4.29,28.6,0.0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 192 entries, 0 to 191
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   DATE    192 non-null    datetime64[ns]
 1   TAVG    192 non-null    float64       
 2   TMAX    192 non-null    float64       
 3   TMIN    192 non-null    float64       
 4   PRCP    192 non-null    float64       
 5   SNOW    192 non-null    float64       
dtypes: datetime64[ns](1), float64(5)
memory usage: 9.1 KB


In [6]:
df.describe()

,TAVG,TMAX,TMIN,PRCP,SNOW
count,192.000000,192.000000,192.000000,192.000000,192.000000
mean,13.373802,17.307656,9.439583,111.375521,66.770833
std,8.664113,8.979786,8.384961,61.889651,161.756101
min,-4.410000,0.080000,-8.900000,9.200000,0.000000
25%,5.407500,9.287500,1.572500,71.000000,0.000000
50%,13.310000,17.595000,9.225000,102.750000,0.000000
75%,21.670000,25.862500,17.615000,138.675000,34.250000
max,27.410000,32.270000,22.970000,481.300000,937.000000


In [7]:
df.isna().any()

DATE    False
TAVG    False
TMAX    False
TMIN    False
PRCP    False
SNOW    False
dtype: bool

In [8]:
fig = go.Figure()


average_temperature = go.Scatter(
    x=df.DATE,
    y=df.TAVG,
    hovertemplate="<i>Date</i>: %{x}"
          "<br><i>Temperature</i>: %{y}°C<br>"
          "<extra></extra>",
    mode='lines',
    name="Average Temperature"
)

max_temperature = go.Scatter(
    x=df.DATE,
    y=df.TMAX,
    hovertemplate="<i>Date</i>: %{x}"
          "<br><i>Temperature</i>: %{y}°C<br>"
          "<extra></extra>",
    mode='lines',
    name="Max Temperature"
    
)

min_temperature = go.Scatter(
    x=df.DATE,
    y=df.TMIN,
    hovertemplate="<i>Date</i>: %{x}"
          "<br><i>Temperature</i>: %{y}°C<br>"
          "<extra></extra>",
    mode='lines',
    name="Min Temperature"
    
)

fig.add_trace(average_temperature)
fig.add_trace(max_temperature)
fig.add_trace(min_temperature)

fig.update_layout(
    title="Average Temperature from 2006-2021",
    xaxis_title="Date",
    yaxis_title="Average Temperature",
)
fig.show()

In [9]:
df

,DATE,TAVG,TMAX,TMIN,PRCP,SNOW
0,2006-01-01,4.93,8.64,1.23,126.8,51.0
1,2006-02-01,2.07,5.78,-1.63,73.1,683.0
2,2006-03-01,6.15,10.33,1.98,20.3,33.0
3,2006-04-01,13.15,18.48,7.82,141.2,3.0
4,2006-05-01,17.27,21.97,12.57,117.5,0.0
...,...,...,...,...,...,...
187,2021-08-01,25.30,28.81,21.80,262.1,0.0
188,2021-09-01,21.27,24.79,17.76,254.8,0.0
189,2021-10-01,16.67,19.93,13.41,133.7,0.0
190,2021-11-01,7.91,11.54,4.29,28.6,0.0


## Weather Affects Crime Rates?

In [16]:
def initializeSpark() -> SparkSession:
    """Create a Spark Session for Streamlit app"""
    conf = SparkConf().setAppName("crime-processor").setMaster("local")
    spark = SparkSession.builder.config(conf=conf).getOrCreate()
    return spark, spark.sparkContext

In [17]:
spark, _ = initializeSpark()

In [18]:
processedSDF = spark.read.load(path=join("..", "data", "NYPD_Complaint_Data_Historic.parquet"), format="parquet")
processedSDF.limit(5).toPandas()

,CMPLNT_NUM,ADDR_PCT_CD,RPT_DT,KY_CD,OFNS_DESC,PD_CD,PD_DESC,CRM_ATPT_CPTD_CD,LAW_CAT_CD,BORO_NM,...,SUSP_RACE,SUSP_SEX,LATITUDE,LONGITUDE,PATROL_BORO,VIC_AGE_GROUP,VIC_RACE,VIC_SEX,CMPLNT_TO,CMPLNT_FR
0,506547392,32,2018-03-30 00:00:00,351,CRIMINAL MISCHIEF,254,"MISCHIEF, CRIMINAL 4, OF MOTOR",COMPLETED,MISDEMEANOR,MANHATTAN,...,None,None,40.810877,-73.941064,PATROL BORO MAN NORTH,25-44,WHITE,F,None,2018-03-29 20:30:00
1,629632833,52,2018-02-07 00:00:00,341,PETIT LARCENY,333,"LARCENY,PETIT FROM STORE-SHOPL",COMPLETED,MISDEMEANOR,BRONX,...,BLACK,F,40.873671,-73.908014,PATROL BORO BRONX,UNKNOWN,UNKNOWN,D,None,2018-02-06 23:15:00
2,787203902,75,2018-11-21 00:00:00,341,PETIT LARCENY,321,"LARCENY,PETIT FROM AUTO",COMPLETED,MISDEMEANOR,BROOKLYN,...,WHITE HISPANIC,F,40.651782,-73.885457,PATROL BORO BKLYN NORTH,UNKNOWN,UNKNOWN,D,2018-11-21 00:20:00,2018-11-21 00:15:00
3,280364018,10,2018-06-10 00:00:00,361,OFFENSES AGAINST PUBLIC ORDER/ADMINISTRATION,639,AGGRAVATED HARASSMENT 2,COMPLETED,MISDEMEANOR,MANHATTAN,...,WHITE HISPANIC,M,40.759310,-73.994706,PATROL BORO MAN SOUTH,18-24,WHITE HISPANIC,F,2018-06-09 21:43:00,2018-06-09 21:42:00
4,985800320,19,2018-11-10 00:00:00,341,PETIT LARCENY,333,"LARCENY,PETIT FROM STORE-SHOPL",COMPLETED,MISDEMEANOR,MANHATTAN,...,BLACK HISPANIC,F,40.764536,-73.970728,PATROL BORO MAN NORTH,UNKNOWN,UNKNOWN,D,2018-11-10 19:45:00,2018-11-10 19:40:00


In [19]:
insideDF = processedSDF\
.select([
    "LOC_OF_OCCUR_DESC",
    year("CMPLNT_FR").alias("CMPLNT_FR_YEAR"),
    month("CMPLNT_FR").alias("CMPLNT_FR_MONTH"),
    dayofmonth("CMPLNT_FR").alias("CMPLNT_FR_DAY"),
    hour("CMPLNT_FR").alias("CMPLNT_FR_HOUR")
]).filter(col("LOC_OF_OCCUR_DESC") != "INSIDE")\
.groupBy(["CMPLNT_FR_YEAR", "CMPLNT_FR_MONTH"])\
.count()\
.sort([col("CMPLNT_FR_YEAR"), col("CMPLNT_FR_MONTH")])\
.toPandas()

In [20]:
insideDF

,CMPLNT_FR_YEAR,CMPLNT_FR_MONTH,count
0,2006,1,12331
1,2006,2,10042
2,2006,3,12100
3,2006,4,7963
4,2006,5,13826
...,...,...,...
187,2021,8,12286
188,2021,9,12641
189,2021,10,13080
190,2021,11,11309


Here we see seasionality that it's hotter in the summer months (June, July, August) and colder in the winter months (December, January, February)

In [ ]:
# https://harris-ippp.github.io/weather.html